In [47]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(100)
def Distance(x,y,type='none'):
    """
    Compute distance.
    """
    x = np.array(x)
    y = np.array(y)
    
    if type == 'none' or type == 'Euclidean':
        return np.linalg.norm(x-y)
    if type == 'Manhattan':
        return np.sum(np.abs(np.abs(x)-np.abs(y)))
    
    if type not in ['none','Euclidean','Manhattan']:
        raise ValueError("Please input correct type!")
        
class SOM_NN:
    """
    Kohonen Neural Network
    """
    def __init__(self,data,nn=30,iterator=1000):
        # Use array_like data。
        data = np.array(data)
        # 自组织网络结构和训练个数
        self.nn = nn
        self.n_sample,self.d = data.shape
        self.som = np.random.rand(self.nn,self.nn,self.d)
        
        # 一些参数
        self.niter = iterator
        self.lamda = self.niter/3.
        self.radius = self.nn/2
        self.learn_rate = 0.1
    
    def find_BMU(self,point):
        """
        Find the Best Matching Unit.
        """
        best_position = [0,0]
        best_distance = 999999
        for i in xrange(self.nn):
            for j in xrange(self.nn):
                distance = Distance(point, self.som[i,j,:],type = 'none')
                if best_distance > distance:
                    best_distance = distance
                    best_position = [i,j]
        
        return best_position
    
    def fit(self,data):
        # 训练数据
        data = np.array(data)
        for t in xrange(self.niter):
            
            index = t%len(data)
            bmu = self.find_BMU(data[index])
            self.ajust_weight(t,bmu,data[index])
            
    def ajust_weight(self,t,bmu,point):
        learning_rate = self.learn_rate * np.exp(-t/self.lamda)
        radius = self.radius * np.exp(-t/self.lamda)
        for i in xrange(self.nn):
            for j in xrange(self.nn):
                dis = Distance([i,j], [bmu[0],bmu[1]],type='Manhattan')
                if dis < radius:
                    self.som[i,j,:] = self.som[i,j,:] + learning_rate * np.exp(-dis**2 / (2* radius**2)) * (point-self.som[i,j,:])
                    
if __name__ == '__main__':
    data = np.random.rand(2000,3)  # 半径衰减有点问题，学习率衰减也有问题
    som = SOM_NN(data,nn=40,iterator=2000)
    %time som.fit(data)
    plt.imshow(som.som)
    plt.savefig('mypic.jpg')

Wall time: 1min 21s
